scaricare hdf view richiede acco gratuito per i download

+ link manuale prisma obsoleti? https://wiki.earthdata.nasa.gov/display/DAS/Toolkit+Downloads  

+ **prisma usa librerie hdf-eos5 v1.13, hdf-eos5 è ora a versione 1.16**  

+ esempio di mmap su hdf5(non hdf-eos5) con julia https://github.com/JuliaIO/HDF5.jl/blob/master/test/mmap.jl  

+ immagine docker per hdf eos lib https://hub.docker.com/r/hdfeos/hdfeos  

+ sdp toolkit: https://wiki.earthdata.nasa.gov/display/DAS/SDP+Toolkit+and+HDF-EOS+EOSDIS++Core+System+Project  

+ tutorial (in c) https://portal.hdfgroup.org/display/HDF5/Learning+the+Basics

+ hdfview: tool da https://support.hdfgroup.org/ftp/HDF5/releases/HDF-JAVA/hdf-java-2.10.1/bin/ (plugin non sembra funzionare con versione più recente)
  plugin per hdf-eos da https://wiki.earthdata.nasa.gov/display/DAS/HDFView+HDF-EOS+Plug-in    
  guida installazione: https://newsroom.gsfc.nasa.gov/sdptoolkit/HDFView/README_HDFView_plugin_v2.10.html    
  
+ documentazione pkg hdf5 https://juliaio.github.io/HDF5.jl/stable/  

+ esempi manipulaz hdf5 in julia https://www.juliabloggers.com/hdf5-in-julia/  

+ pkg R per importare dati prisma l1/l2 https://lbusett.github.io/prismaread/  
+ https://portal.hdfgroup.org/display/support/HDF5%201.12.0

In [4]:
# docu package hdf5 per julia https://juliaio.github.io/HDF5.jl/stable/
# using Pkg
# pkg.add("HDF5")
using HDF5
using DataFrames

#errori posson lasciare aperti file-> file inseriti in struct dinamica per poi esser facilmente chiusi tutti
openfiles = Dict()
counter = 0
# funzione che chiude tutti i file aperti
function closeall(dict)
    n = 0
    for (key, value) in dict
        close(pop!(dict, key))
        n=n+1;
    end
    counter = 0
    println("chiuso $n file")
end

closeall (generic function with 1 method)

apertura file con h5open(filename::string, mode::string) -> file   
mode:  

| mode | Meaning |
|---|---|
|"r"|read-only|
|"r+"|read-write, preserving any existing contents|
|"cw"|read-write, create file if not existing, preserve existing contents|
|"w"|read-write, destroying any existing contents (if any)|

In [5]:
testfilename = "test.h5"
fid = h5open(testfilename, "w") # crea file hdf5 vuoto (o lo apre e basta se già esiste)

close(fid)


In [6]:
close(fid)
rm(testfilename) #deleta file di prova

In [14]:
mutable struct lnode
    name::String
    leftmost::Union{lnode,Nothing}
    parent::Union{lnode,Nothing}
    rightsib::Union{lnode,Nothing}
    lnode(v::String) = (x = new(); x.leftmost = nothing; x.rightsib=nothing;x.parent=nothing; x.name=v; x)
end
# per lista dopiam collegata basta aggiungere un campo parent::lnode 

mutable struct treeList
    first::lnode
    List(root::lnode) = List(nodo)    
end

#struttura albero leftmost child/right sibling per alberi non binari

In [25]:
function getrightmost(parent::lnode)
    temp = parent
    if parent.leftmost != nothing
        temp = parent.leftmost
        while (temp.rightsib != nothing)
            temp = temp.rightsib
        end
        temp
    else
        throw("input has no parent")        
    end
end

function addchild!(parent::lnode, name::String)
    new = lnode(name)
    new.parent = parent
    if(parent.leftmost!=nothing)
        temp = getrightmost(parent)
        temp.rightsib = new
        new
    else
        parent.rightmost = new
        new
    end
end
    
    

function treeToDf(parent::Union{lnode,Nothing}, pid, counter)#prende albero, ritorna df
        #si aggiunge
        if pid==0
            pid=nothing#per root
        end
        myid=counter
        counter = counter+1
        mydf = DataFrame(id=myid,name=lnode.name,parent=pid)
        #lancia view su figlio        
        if (parent.leftmost)!=nothing
            tempdf, counter = myView(parent.leftmost,myid,counter)
            append!(mydf,tempdf)
        end
        #lancia view sul fratello
        if (parent.rightsib)!=nothing
            tempdf, counter = myView(parent.leftmost,pid,counter)
            append!(mydf,tempdf)
        end
        #ritorna df e counter
        mydf, counter
    end
    

function viewfile(x)
    println("call viewfile") 
    myRoot = lnode("HDF5 FILE")
    viewgroup(x,myRoot)
    
end
    
function viewgroup(g,myname)
    println("call viewgroup on $g")
    #nome
    #println(g)
    
    # attributi
    attributi = attrs(g)
    if isempty(names(attributi))
        println("niente attributi")
    else        
        viewattributes(attributi)
    end
    #attrs(obj) ritorna attributi oggetto
    nomi = names(g)
    # names(x::Union{HDF5Group,HDF5File}) ritorna array di nomi dei membri di obj
    println("nomi: $nomi")
    #view su oggetti contenuti
    for nome in nomi
        i = g[nome]
        ti = typeof(i)
        
        println("trovato sotto oggetto di nome: $nome e tipo $ti")
        if ti==HDF5Group            
            viewgroup(i)
        elseif ti==HDF5Dataset
            viewdata(i)
        else
            println(ti)
        end
    end
end

function viewdata(d)
    println("call viewdata on $d")
    
    #attributi
    attributi = attrs(d)
    if isempty(names(attributi))
        println("niente attributi")
    else        
        viewattributes(attributi)
    end
    
    # nome
    #println(d)
    #1 dataset mappabile?
    if ismmappable(d)
        println("dataset mmappabile")
    else
        println("dataset non mmappabile")
    end
    #mappatura dataset con readmmap() aumenta rpestazioni per letture numerose di un ds
    
    # datatype
    println("tipo dati: $(typeof(read(d)))")
    
    # dataspace (dimensioni )
    println("dimensioni: $(size(d))")
end 

function viewattributes(atts::HDF5.HDF5Attributes)
    println("call viewattributes on $atts")
    nomi = names(atts)
    for nome in nomi
        content = read(atts, nome)
        println("attributo di nome: $nome ; tipo: $(typeof(content))")
        println("contenuto: $content")
    end
end

myView (generic function with 1 method)

apre file
1. trova attributi file
2. per ogni attributo
    1. stampa nome, tipo, contenuto
2. trova sotto oggetti
3. per ogni sotto oggetto
    1. stampa nome, tipo
    2. trova attributi
    3. per ogni attrib
        1. stampa nome, tipo, contenuto
    5. se l'oggetto è un dataset, ne stampa le dimensioni ed il tipo di dati
    4. trova sotto oggetti (sse oggetto è gruppo, dataset non han sotto oggetti)    
    5. per ogni sotto oggetto
        1. vedi 4.

viewattributes (generic function with 1 method)

In [ ]:
function viewfile(file, dataset)
    println("call viewfile")
    dataset = DataFrame(id=1,name=file,parent = missing)
   #=for i in x 
        if typeof(i)==HDF5Group
            viewgroup(i)
        else
            println(typeof(i))
        end
    end=#
    viewgroup(x,dataset,1)
    #dump(x)
end
    
function viewgroup(g,df,parentid)
    println("call viewgroup on $g")
    #nome
    #println(g)
    
    # attributi
    attributi = attrs(g)
    if isempty(names(attributi))
        println("niente attributi")
    else        
        viewattributes(attributi)
    end
    #attrs(obj) ritorna attributi oggetto
    nomi = names(g)
    # names(x::Union{HDF5Group,HDF5File}) ritorna array di nomi dei membri di obj
    println("nomi: $nomi")
    #view su oggetti contenuti
    for nome in nomi
        i = g[nome]
        ti = typeof(i)
        println("trovato sotto oggetto di nome: $nome e tipo $ti")
        if ti==HDF5Group            
            viewgroup(i)
        elseif ti==HDF5Dataset
            viewdata(i)
        else
            println(ti)
        end
    end
end

function viewdata(d)
    println("call viewdata on $d")
    
    #attributi
    attributi = attrs(d)
    if isempty(names(attributi))
        println("niente attributi")
    else        
        viewattributes(attributi)
    end
    
    # nome
    #println(d)
    #1 dataset mappabile?
    if ismmappable(d)
        println("dataset mmappabile")
    else
        println("dataset non mmappabile")
    end
    #mappatura dataset con readmmap() aumenta rpestazioni per letture numerose di un ds
    
    # datatype
    println("tipo dati: $(typeof(read(d)))")
    
    # dataspace (dimensioni )
    println("dimensioni: $(size(d))")
end 

function viewattributes(atts::HDF5.HDF5Attributes)
    println("call viewattributes on $atts")
    nomi = names(atts)
    for nome in nomi
        content = read(atts, nome)
        println("attributo di nome: $nome ; tipo: $(typeof(content))")
        println("contenuto: $content")
    end
end
     

In [8]:
#apro in lettura un file sample preso da hdfviewer, faccio qualcosa, chiudo file
#inputfile = "OMI-Aura_.he5"
inputfile = "data/PRS_L2D_STD_20200708101329_20200708101333_0001/PRS_L2D_STD_20200708101329_20200708101333_0001.he5"
println("ishdf5 ? $(ishdf5(inputfile))") #ishdf5 prende stringa (nome file) ritorna se file è hdf5
file2 = h5open(inputfile, "r")
openfiles["file $counter"]=file2
counter = counter+1
#foo(file2)
viewfile(file2)
closeall(openfiles)

ishdf5 ? true
call viewfile
call viewgroup on HDF5 data file: data/PRS_L2D_STD_20200708101329_20200708101333_0001/PRS_L2D_STD_20200708101329_20200708101333_0001.he5
call viewattributes on HDF5.HDF5Attributes(HDF5 data file: data/PRS_L2D_STD_20200708101329_20200708101333_0001/PRS_L2D_STD_20200708101329_20200708101333_0001.he5)
attributo di nome: Acquisition_Purpose ; tipo: String
contenuto: NOT SPECIAL PRODUCT
attributo di nome: Acquisition_Size ; tipo: String
contenuto: 30 Km
attributo di nome: Acquisition_Station ; tipo: String
contenuto: CNM
attributo di nome: Acquisition_Type ; tipo: String
contenuto: EARTH OBSERVATION
attributo di nome: Atm_LutGeomInfo_RelativeAzimuth ; tipo: Array{String,1}
contenuto: ["RAA_050", "RAA_085"]
attributo di nome: Atm_LutGeomInfo_SunZenith ; tipo: Array{String,1}
contenuto: ["SZA_020", "SZA_035"]
attributo di nome: Atm_LutGeomInfo_ViewZenith ; tipo: Array{String,1}
contenuto: ["VZA_000", "VZA_010"]
attributo di nome: Atm_Lut_version ; tipo: String
cont

contenuto: UInt8[0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x0


attributo di nome: PAN_ACQ ; tipo: UInt8
contenuto: 3
attributo di nome: PAN_Corrupted_Frame_Percentage ; tipo: String
contenuto: 00.00 %
attributo di nome: PAN_HGRP ; tipo: UInt8
contenuto: 1
attributo di nome: PAN_HYP_ACT_RESIDUAL_m ; tipo: Float32
contenuto: 189.36986
attributo di nome: PAN_HYP_ALT_RESIDUAL_m ; tipo: Float32
contenuto: 2.950063
attributo di nome: PAN_HYP_START_SYNC_FRAME ; tipo: UInt32
contenuto: 97
attributo di nome: PAN_HYP_START_SYNC_SUBFRAME ; tipo: UInt32
contenuto: 4
attributo di nome: PAN_HYP_STOP_SYNC_FRAME ; tipo: UInt32
contenuto: 1097
attributo di nome: PAN_HYP_STOP_SYNC_SUBFRAME ; tipo: UInt32
contenuto: 0
attributo di nome: PAN_Int_Time ; tipo: Float32
contenuto: 0.00071
attributo di nome: PE_Gain_SWIR ; tipo: Array{UInt8,1}
contenuto: UInt8[0x01, 0x01, 0x01, 0x01, 0x00, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01

contenuto: UInt8[0x01 0x01 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x0

contenuto: UInt8[0x01 0x01 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x0

contenuto: Int32[0]
attributo di nome: GCP_Min_Correlation_threshold ; tipo: UInt32
contenuto: 0
attributo di nome: GCP_Overall_Correlation_score ; tipo: Int32
contenuto: 0
attributo di nome: GCP_Use_Flag ; tipo: UInt8
contenuto: 0
attributo di nome: GCP_Used ; tipo: Int32
contenuto: 0
attributo di nome: Geocoding_RMS_Error ; tipo: Float32
contenuto: 0.0
nomi: String[]
trovato sotto oggetto di nome: PRS_L2D_HCO e tipo HDF5Group
call viewgroup on HDF5 group: /HDFEOS/SWATHS/PRS_L2D_HCO (file: data/PRS_L2D_STD_20200708101329_20200708101333_0001/PRS_L2D_STD_20200708101329_20200708101333_0001.he5)
niente attributi
nomi: ["Data Fields", "Geocoding Model", "Geolocation Fields", "Geometric Fields"]
trovato sotto oggetto di nome: Data Fields e tipo HDF5Group
call viewgroup on HDF5 group: /HDFEOS/SWATHS/PRS_L2D_HCO/Data Fields (file: data/PRS_L2D_STD_20200708101329_20200708101333_0001/PRS_L2D_STD_20200708101329_20200708101333_0001.he5)
niente attributi
nomi: ["SWIR_Cube", "SWIR_PIXEL_L2_ERR_MATR

dataset mmappabile
tipo dati: Array{UInt16,2}
dimensioni: (7518, 7404)
trovato sotto oggetto di nome: PIXEL_L2_ERR_MATRIX e tipo HDF5Dataset
call viewdata on HDF5 dataset: /HDFEOS/SWATHS/PRS_L2D_PCO/Data Fields/PIXEL_L2_ERR_MATRIX (file: data/PRS_L2D_STD_20200708101329_20200708101333_0001/PRS_L2D_STD_20200708101329_20200708101333_0001.he5 xfer_mode: 0)
niente attributi
dataset mmappabile
tipo dati: Array{UInt8,2}
dimensioni: (7518, 7404)
trovato sotto oggetto di nome: Geocoding Model e tipo HDF5Group
call viewgroup on HDF5 group: /HDFEOS/SWATHS/PRS_L2D_PCO/Geocoding Model (file: data/PRS_L2D_STD_20200708101329_20200708101333_0001/PRS_L2D_STD_20200708101329_20200708101333_0001.he5)
call viewattributes on HDF5.HDF5Attributes(HDF5 group: /HDFEOS/SWATHS/PRS_L2D_PCO/Geocoding Model (file: data/PRS_L2D_STD_20200708101329_20200708101333_0001/PRS_L2D_STD_20200708101329_20200708101333_0001.he5))
attributo di nome: ERR_BIAS ; tipo: Float32
contenuto: -1.0
attributo di nome: ERR_RAND ; tipo: Floa

call viewdata on HDF5 dataset: /Info/Ancillary/PVSdata/Wgs84_pos_z (file: data/PRS_L2D_STD_20200708101329_20200708101333_0001/PRS_L2D_STD_20200708101329_20200708101333_0001.he5 xfer_mode: 0)
niente attributi
dataset mmappabile
tipo dati: Array{Float32,2}
dimensioni: (1, 1120)
trovato sotto oggetto di nome: Wgs84_vel_x e tipo HDF5Dataset
call viewdata on HDF5 dataset: /Info/Ancillary/PVSdata/Wgs84_vel_x (file: data/PRS_L2D_STD_20200708101329_20200708101333_0001/PRS_L2D_STD_20200708101329_20200708101333_0001.he5 xfer_mode: 0)
niente attributi
dataset mmappabile
tipo dati: Array{Float32,2}
dimensioni: (1, 1120)
trovato sotto oggetto di nome: Wgs84_vel_y e tipo HDF5Dataset
call viewdata on HDF5 dataset: /Info/Ancillary/PVSdata/Wgs84_vel_y (file: data/PRS_L2D_STD_20200708101329_20200708101333_0001/PRS_L2D_STD_20200708101329_20200708101333_0001.he5 xfer_mode: 0)
niente attributi
dataset mmappabile
tipo dati: Array{Float32,2}
dimensioni: (1, 1120)
trovato sotto oggetto di nome: Wgs_84_vel_z e

chiuso 1 file


In [119]:
openfiles

Dict{Any,Any}()

## R
https://shekeine.github.io/modis/2014/08/27/HDF_to_multi-band_geotiff

https://hakimabdi.com/blog/working-with-hdf-eos-files-in-r-convert-project-and-mosaicù

installa gdalUtils package
installa gdalUtilities package

In [11]:
using DataFrames
df = DataFrame(id = 1, name = "HDF5FILE", parent = missing)

,id,name,parent
,Int64,String,Missing
1,1,HDF5FILE,missing


In [16]:
c = nothing
c!=nothing

false

In [17]:
c == nothing

true

In [18]:
c=0
c!=nothing

true

In [19]:
myarr= [1,2,3]
append!(myarr, 4)

4-element Array{Int64,1}:
 1
 2
 3
 4

In [21]:
function tryDFLoop(names)
    df = nothing
    counter = 0
    for name in names
        tempdf = DataFrame(id = counter, nome = name)
        counter = counter+1
        if df!=nothing
            append!(df,tempdf)
        else
            df = tempdf
        end
    end
    df
end

tryDFLoop (generic function with 1 method)

In [23]:
nas = ["primo","secondo","terzo"]
tryDFLoop(nas)

,id,nome
,Int64,String
1,0,primo
2,1,secondo
3,2,terzo
